# Elementos da versão

Antes mesmo de falar do projeto, ao longo do desenvolvimento, vamos explicitando qual o objetivo de cada versão de trabalho do arquivo principal.

Tendo em vista diferença dos dados de mortalidade por doenças evitáveis conforme publicado pelo IEPS, o objetivo desta v3 é recriar o dataset da v2, mas utilizar os dados do Censo de 2010.

O IEPS apresenta os seguintes valores de mortalidade de doenças evitáveis para o Brasil:

* 2010 - 99
* 2020 - 89

Ainda que na documentação para criação das variáveis o IEPS fale de dados ajustados por ano, não ficou claro que usem outros dados além dos do Censo 2010.

O dataset desta v.2 deverá ter as seguintes características:

* setting: Brasil [poderemos encontrar um sufixo a Brasil que permita a sua subclassificação e carga deste dataset junto a outros]
* dimension: UF
* subgroup: municípios
* estimate: mort-avoid por município
* setting-average: mort-avoid por Brasil

# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




# Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [4]:
ocde_evit = pd.read_csv('downloads_outros/cid10_evit_ocde.csv')
ocde_evit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


In [76]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_clean/dobr_clean_2010_19a22.parquet')


In [77]:
dobr_evit_merged_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446922 entries, 0 to 4446921
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   CAUSABAS   object
 1   CODMUNRES  object
 2   IDADE      int64 
 3   ANO_OBITO  object
dtypes: int64(1), object(3)
memory usage: 135.7+ MB


In [82]:
# Filtrar apenas anos de 2010 e 2020
dobr_evit_merged_temp01 = dobr_evit_merged_temp[dobr_evit_merged_temp['ANO_OBITO'].isin(['_2010', '_2020'])]
dobr_evit_merged_temp01['ANO_OBITO'].unique()

array(['_2020', '_2010'], dtype=object)

In [84]:
causas_unicas = dobr_evit_merged_temp01['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10452 entries, 0 to 10451
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  10452 non-null  object
dtypes: object(1)
memory usage: 81.8+ KB


In [85]:
# Fazer merge da lista de causas_unicas com a lista de causas evitáveis

df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit
column_b = 'cid_pai'
column_return = 'avoid_flag'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['classificacao'] = df_a.apply(verificar_correspondencia, axis=1)

print(df_a)

       CAUSABAS        classificacao
0          B342   causa não evitável
1          F102  Avoidable mortality
2          A419   causa não evitável
3          E149  Avoidable mortality
4          E162   causa não evitável
...         ...                  ...
10447  W041      Avoidable mortality
10448  V155      Avoidable mortality
10449  R456       causa não evitável
10450  L303       causa não evitável
10451  D040       causa não evitável

[10452 rows x 2 columns]


In [86]:
# Merge dobr_evit_merged_temp01 and df_a by CAUSABAS
dobr_evit_merged_temp02 = pd.merge(dobr_evit_merged_temp01, df_a, on='CAUSABAS', how='left')

# Display the first few rows of the merged dataframe
dobr_evit_merged_temp02.head()

,CAUSABAS,CODMUNRES,IDADE,ANO_OBITO,classificacao
0,B342,_311760,74,_2020,causa não evitável
1,F102,_315140,63,_2020,Avoidable mortality
2,A419,_330455,74,_2020,causa não evitável
3,B342,_310620,57,_2020,causa não evitável
4,E149,_250750,69,_2020,Avoidable mortality


In [87]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp02.groupby('classificacao').count()

,CAUSABAS,CODMUNRES,IDADE,ANO_OBITO
classificacao,,,,
Avoidable mortality,978472,978472,978472,978472
causa não evitável,667020,667020,667020,667020


In [88]:
# Manter na base só dados de CID evitáveis
dobr_evit_merged_temp03 = dobr_evit_merged_temp02[dobr_evit_merged_temp02['classificacao']=='Avoidable mortality']


#### Resultados do merge de cid_avoid e dobr

In [89]:
# Salvar arquivo final de DOBR
dobr_evit_merged_temp03.to_parquet('files_clean/dobr_evit_merged_v3.parquet')


In [90]:
# Contador

shape_origin = dobr_evit_merged_temp.CAUSABAS.count()
shape_filt = dobr_evit_merged_temp03.CAUSABAS.count()
dif = shape_origin - shape_filt


print_y(f"O número de óbitos após a limpeza e que constituía a base de dados de DO era de {shape_origin} óbitos")
print_y(f"O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de {dif} óbitos")
print_y(f"O número de óbitos por CID EVITÁVEIS é de {shape_filt} óbitos")

 ==> O número de óbitos após a limpeza e que constituía a base de dados de DO era de 4446922 óbitos
 ==> O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de 3468450 óbitos
 ==> O número de óbitos por CID EVITÁVEIS é de 978472 óbitos


### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [17]:
# Verificar colunas de população no arquivo censo_pop_clean
censo_mun_temp = pd.read_parquet('files_clean/censo_pop_mun.parquet')
censo_mun_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11140 entries, 0 to 11139
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ano          11140 non-null  object
 1   uf_co        11140 non-null  object
 2   UF           11140 non-null  object
 3   mun_cod_sim  11140 non-null  object
 4   mun_no       11140 non-null  object
 5   pop_total    11140 non-null  int64 
 6   pop_under74  11140 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 609.3+ KB


In [18]:
censo_mun_temp.head()

,Ano,uf_co,UF,mun_cod_sim,mun_no,pop_total,pop_under74
0,2010,_11,RO,_110001,Alta Floresta D'Oeste,24392,23948
1,2010,_11,RO,_110002,Ariquemes,90353,89037
2,2010,_11,RO,_110003,Cabixi,6313,6180
3,2010,_11,RO,_110004,Cacoal,78574,76947
4,2010,_11,RO,_110005,Cerejeiras,17029,16646


In [20]:
# Para verificar o modelo IPES, vamos fazer o merge apenas com dados do Censo de 2010
censo_mun_temp01 = censo_mun_temp.loc[censo_mun_temp['Ano']=='2010']
censo_mun_temp01.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5570 entries, 0 to 5569
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ano          5570 non-null   object
 1   uf_co        5570 non-null   object
 2   UF           5570 non-null   object
 3   mun_cod_sim  5570 non-null   object
 4   mun_no       5570 non-null   object
 5   pop_total    5570 non-null   int64 
 6   pop_under74  5570 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 348.1+ KB


In [ ]:
# Criar um campo string que concatene ANO, CODMUN_SIM, Sexo e cor # Não é preciso fazer o merge com o ano, pois iremos usar o Censo 2010 para todos os anos
# censo_pop_mun_4merge['POP_MERGE'] = '_' + censo_pop_mun_4merge['Ano'].astype(str) + censo_pop_mun_4merge['mun_cod_sim']

# censo_pop_mun_4merge.head()

In [91]:
# Exibir arquivo dobr_clean
dobr_merged_temp = pd.read_parquet('files_clean/dobr_evit_merged_v3.parquet')
dobr_merged_temp.head()


,CAUSABAS,CODMUNRES,IDADE,ANO_OBITO,classificacao
1,F102,_315140,63,_2020,Avoidable mortality
4,E149,_250750,69,_2020,Avoidable mortality
11,I619,_330455,66,_2020,Avoidable mortality
13,W849,_330455,58,_2020,Avoidable mortality
14,I10,_330455,68,_2020,Avoidable mortality


In [93]:
dobr_merged_temp.columns

Index(['CAUSABAS', 'CODMUNRES', 'IDADE', 'ANO_OBITO', 'classificacao'], dtype='object')

########## Este bloco de código a seguir retira o CAUSABAS

In [94]:
# Agrupar por CODMUNRES e somar os registros, não contabilizando mais por CAUSABAS

col_group= [
            # 'CAUSABAS', 
            'CODMUNRES', 'ANO_OBITO']
dobr_merged_temp01 = dobr_merged_temp.groupby(col_group)['CAUSABAS'].count().reset_index()

# Renomear o campo CAUSABAS para CONTADOR
dobr_merged_temp01.rename(columns={'CAUSABAS':'CONTADOR'}, inplace=True)

# Exibir as primeiras linhas do dataframe resultante
dobr_merged_temp01.head()

,CODMUNRES,ANO_OBITO,CONTADOR
0,_110000,_2010,20
1,_110000,_2020,7
2,_110001,_2010,58
3,_110001,_2020,68
4,_110002,_2010,217


In [95]:
# Verificar o total de óbitos da base antes e depois do groupby (após o groupby foi feita uma totalidade por soma das contagens por registro)
print(dobr_merged_temp['CAUSABAS'].count())
print(dobr_merged_temp01['CONTADOR'].sum())

978472
978472


In [96]:
# Verificação dos nomes das colunas
print(dobr_merged_temp01.columns)
print(censo_mun_temp01.columns)

Index(['CODMUNRES', 'ANO_OBITO', 'CONTADOR'], dtype='object')
Index(['Ano', 'uf_co', 'UF', 'mun_cod_sim', 'mun_no', 'pop_total',
       'pop_under74'],
      dtype='object')


In [97]:
# Definir campos de censo_pop para merge
censo_mun_temp02 = censo_mun_temp01[['mun_cod_sim','pop_under74','UF','mun_no']]

# Realizar o merge dos dataframes
dobr_merged_temp02 = pd.merge(dobr_merged_temp01, censo_mun_temp02, left_on=['CODMUNRES'], right_on=['mun_cod_sim'], how='left')

# Exibir as primeiras linhas do dataframe resultante
dobr_merged_temp02.head()

,CODMUNRES,ANO_OBITO,CONTADOR,mun_cod_sim,pop_under74,UF,mun_no
0,_110000,_2010,20,NaN,NaN,NaN,NaN
1,_110000,_2020,7,NaN,NaN,NaN,NaN
2,_110001,_2010,58,_110001,23948.0,RO,Alta Floresta D'Oeste
3,_110001,_2020,68,_110001,23948.0,RO,Alta Floresta D'Oeste
4,_110002,_2010,217,_110002,89037.0,RO,Ariquemes


In [98]:
# Filtrar registros onde pop_under74 não é nulo
dobr_merged_temp03 = dobr_merged_temp02.dropna(subset=['pop_under74']).reset_index(drop=True)

# Exibir os registros filtrados
dobr_merged_temp03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11117 entries, 0 to 11116
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CODMUNRES    11117 non-null  object 
 1   ANO_OBITO    11117 non-null  object 
 2   CONTADOR     11117 non-null  int64  
 3   mun_cod_sim  11117 non-null  object 
 4   pop_under74  11117 non-null  float64
 5   UF           11117 non-null  object 
 6   mun_no       11117 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 608.1+ KB


In [99]:
# Verificar o total de óbitos da base depos da operação acima
print(dobr_merged_temp03['CONTADOR'].sum())

976505


##### Arquivo tratado final

In [100]:
# Salvar arquivo
dobr_merged_temp03.to_parquet('files_clean/dobr_merged_v3.parquet')

In [101]:
# Resultados

obito_origin = dobr_merged_temp.CAUSABAS.count()
obito_final = dobr_merged_temp03.CONTADOR.sum()
dif = obito_origin - obito_final

print_y(f" O número de óbitos após merge com tabela de CIDs evitáveis era de {obito_origin} óbitos ")

print_y(f" Foram excluídos, por apresentar problemas para carga de informações de município, {dif} óbitos ")

print_y(f" O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de {obito_final} óbitos ")


# # Filtrar registros onde pop_under74 é null
# dobr_merged_pop_nulos = dobr_merged_pop[dobr_merged_pop['pop_under74'].isnull()]

# # Exibir os registros filtrados
# dobr_merged_pop_nulos['obito_count'].sum()

 ==>  O número de óbitos após merge com tabela de CIDs evitáveis era de 978472 óbitos 
 ==>  Foram excluídos, por apresentar problemas para carga de informações de município, 1967 óbitos 
 ==>  O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de 976505 óbitos 


# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 3 - IEPS BR/uf/Mun


O primeiro dataset que iremos construir tem as seguintes características:

* setting: Brasil
* dimension: UF
* subgroup: município

In [ ]:
# # Mostrar colunas do template do HEAT

# import requests
# import openpyxl

# # Baixando o arquivo
# url = 'https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm'
# response = requests.get(url)

# # Salvando o arquivo localmente (você pode escolher outro nome)
# with open('arquivo_excel.xlsm', 'wb') as f:
#     f.write(response.content)

# # Abrindo o arquivo Excel
# workbook = openpyxl.load_workbook('arquivo_excel.xlsm')

# # Selecionando a primeira aba
# sheet = workbook.template
# sheet = workbook['template']

# # Mostrar conteúdo da aba 'template'
# for row in sheet.iter_rows(values_only=True):
#     print(row)



('setting', 'date', 'source', 'indicator_abbr', 'indicator_name', 'dimension', 'subgroup', 'estimate', 'se', 'ci_lb', 'ci_ub', 'population', 'note', 'setting_average', 'iso3', 'favourable_indicator', 'indicator_scale', 'ordered_dimension', 'subgroup_order', 'reference_subgroup')


In [102]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files_clean/dobr_merged_v3.parquet')
heatds_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11117 entries, 0 to 11116
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CODMUNRES    11117 non-null  object 
 1   ANO_OBITO    11117 non-null  object 
 2   CONTADOR     11117 non-null  int64  
 3   mun_cod_sim  11117 non-null  object 
 4   pop_under74  11117 non-null  float64
 5   UF           11117 non-null  object 
 6   mun_no       11117 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 608.1+ KB


In [103]:
heatds_temp.head()

,CODMUNRES,ANO_OBITO,CONTADOR,mun_cod_sim,pop_under74,UF,mun_no
0,_110001,_2010,58,_110001,23948.0,RO,Alta Floresta D'Oeste
1,_110001,_2020,68,_110001,23948.0,RO,Alta Floresta D'Oeste
2,_110002,_2010,217,_110002,89037.0,RO,Ariquemes
3,_110002,_2020,266,_110002,89037.0,RO,Ariquemes
4,_110003,_2010,15,_110003,6180.0,RO,Cabixi


In [104]:
heatds_temp.columns

Index(['CODMUNRES', 'ANO_OBITO', 'CONTADOR', 'mun_cod_sim', 'pop_under74',
       'UF', 'mun_no'],
      dtype='object')

In [38]:
# # Agregar os dados sem CID e gerar colunas de soma de obito_count e pop_under74
# heat_semCID = heat_.groupby(['UF', 'mun_nome', 'CODMUNRES', 'SEXO', 'RACACOR', 'ANO']).agg({
#     'obito_count': 'sum',
#     'pop_under74': 'sum'
# }).reset_index()



In [105]:
# Gerar o indicador de mortalidade (por 100 mil habitantes) e exibir as primeiras linhas do DataFrame
heatds_temp01 = heatds_temp.copy()

heatds_temp01['mort_ind_mun'] = (heatds_temp01['CONTADOR']/heatds_temp01['pop_under74'])*100000

# Exibir as primeiras linhas do dataframe resultante
heatds_temp01.head()

,CODMUNRES,ANO_OBITO,CONTADOR,mun_cod_sim,pop_under74,UF,mun_no,mort_ind_mun
0,_110001,_2010,58,_110001,23948.0,RO,Alta Floresta D'Oeste,242.191415
1,_110001,_2020,68,_110001,23948.0,RO,Alta Floresta D'Oeste,283.948555
2,_110002,_2010,217,_110002,89037.0,RO,Ariquemes,243.718903
3,_110002,_2020,266,_110002,89037.0,RO,Ariquemes,298.752204
4,_110003,_2010,15,_110003,6180.0,RO,Cabixi,242.718447


In [106]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp01.describe()

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,CONTADOR,pop_under74,mort_ind_mun
count,11117.000000,1.111700e+04,1.111700e+04
mean,87.838895,3.332522e+04,inf
std,506.484000,1.964664e+05,NaN
min,1.000000,0.000000e+00,1.264063e+01
25%,12.000000,5.060000e+03,1.999429e+02
50%,26.000000,1.062000e+04,2.534630e+02
75%,59.000000,2.273500e+04,3.041961e+02
max,28299.000000,1.087806e+07,inf


In [116]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
# registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun'])]
inf_count = np.isinf(heatds_temp01['mort_ind_mun']).sum()

# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
# registros_inf

# Exportar registros de indicador de mortalidade infinitos [inf]
# registros_inf.to_csv('exportar/relatorio_reg_mort_ind_inf.csv')

Número de valores infinitos em mort_ind_mun: 5


In [124]:
# Mostrar registros com números infinitos
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun'])]
registros_inf.head()

,CODMUNRES,ANO_OBITO,CONTADOR,mun_cod_sim,pop_under74,UF,mun_no,mort_ind_mun
448,_150475,_2020,39,_150475,0.0,PA,Mojuí dos Campos,inf
8993,_421265,_2020,28,_421265,0.0,SC,Pescaria Brava,inf
9196,_422000,_2020,63,_422000,0.0,SC,Balneário Rincão,inf
9829,_431454,_2020,3,_431454,0.0,RS,Pinto Bandeira,inf
10302,_500627,_2020,9,_500627,0.0,MS,Paraíso das Águas,inf


In [117]:
heatds_temp01.CONTADOR.sum()

976505

In [127]:
# # Limpar a base de registros de indicador de mortalidade infinitos [inf]
heatds_temp02 = heatds_temp01.copy()

# # Substituir valores infinitos por NaN
# heatds_temp02['mort_ind_mun'].replace([np.inf, -np.inf], np.nan, inplace=True)
heatds_temp02['mort_ind_mun_'] = heatds_temp02['mort_ind_mun'].replace([np.inf, -np.inf], np.nan)



In [136]:
# # Renomear colunas
heatds_temp03 = heatds_temp02.drop(columns=['mort_ind_mun'])

heatds_temp03.rename(columns={'mort_ind_mun_': 'mort_ind_mun'}, inplace=True)

heatds_temp03.head()

,CODMUNRES,ANO_OBITO,CONTADOR,mun_cod_sim,pop_under74,UF,mun_no,mort_ind_mun
0,_110001,_2010,58,_110001,23948.0,RO,Alta Floresta D'Oeste,242.191415
1,_110001,_2020,68,_110001,23948.0,RO,Alta Floresta D'Oeste,283.948555
2,_110002,_2010,217,_110002,89037.0,RO,Ariquemes,243.718903
3,_110002,_2020,266,_110002,89037.0,RO,Ariquemes,298.752204
4,_110003,_2010,15,_110003,6180.0,RO,Cabixi,242.718447


In [137]:
# Contar registros nulos na coluna mort_ind_mun
nulos_mort_ind_mun = heatds_temp02['mort_ind_mun'].isnull().sum()
print(f"Número de registros nulos em mort_ind_mun: {nulos_mort_ind_mun}")

Número de registros nulos em mort_ind_mun: 0


In [139]:
# Excluir registros com NaN na coluna mort_ind_mun
heatds_temp03.dropna(subset=['mort_ind_mun'], inplace=True)

# Verificar as primeiras linhas do DataFrame atualizado
heatds_temp03.head()

,CODMUNRES,ANO_OBITO,CONTADOR,mun_cod_sim,pop_under74,UF,mun_no,mort_ind_mun
0,_110001,_2010,58,_110001,23948.0,RO,Alta Floresta D'Oeste,242.191415
1,_110001,_2020,68,_110001,23948.0,RO,Alta Floresta D'Oeste,283.948555
2,_110002,_2010,217,_110002,89037.0,RO,Ariquemes,243.718903
3,_110002,_2020,266,_110002,89037.0,RO,Ariquemes,298.752204
4,_110003,_2010,15,_110003,6180.0,RO,Cabixi,242.718447


In [ ]:
# Renomear coluna

In [140]:
# Comparar os valores antes e depois de excluir valores infinitos de indicador de mortalidade
origem = heatds_temp01.CONTADOR.sum()
final = heatds_temp03.CONTADOR.sum()
dif = final - origem
print_y(f'Antes da criação do indicador de mortalidade, tinhamos registro de {origem} mortes, dada a limpeza de {dif} registros, temos o total final de registros: {final}')

 ==> Antes da criação do indicador de mortalidade, tinhamos registro de 976505 mortes, dada a limpeza de -142 registros, temos o total final de registros: 976363


In [74]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp03.describe()

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,CONTADOR,pop_under74,mort_ind_mun
count,27817.000000,2.781700e+04,2.781700e+04
mean,91.078405,3.329709e+04,inf
std,510.470539,1.963769e+05,NaN
min,1.000000,0.000000e+00,1.264063e+01
25%,13.000000,5.051000e+03,2.152890e+02
50%,27.000000,1.059800e+04,2.676182e+02
75%,62.000000,2.272300e+04,3.202254e+02
max,28299.000000,1.087806e+07,inf


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [195]:
heat_ds_treat1.columns

Index(['CODMUNRES', 'ANO_OBITO', 'POP_MERGE', 'CONTADOR', 'pop_under74', 'UF',
       'mun_no', 'mort_ind_mun'],
      dtype='object')

In [ ]:
# # Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
# dobr_grouped_uf = heat_ds_treat2.groupby(['UF',
#                                              'ANO_OBITO',
#                                             #  'SEXO',
#                                             #  'RACACOR'
#                                              ]).agg({'CONTADOR': 'sum',
#                                                     'pop_under74': 'sum',
#                                                     'mort_ind_mun': 'mean'
#                                                 }).reset_index()
# dobr_grouped_uf.rename(columns={'mort_ind_mun': 'mort_ind_uf',
#                                 'CONTADOR': 'obito_count_uf',
#                                 'pop_under74': 'pop_under74_uf',
#                                 }, inplace=True)


# # Exibir as primeiras linhas do DataFrame resultante
# dobr_grouped_uf.head()

In [ ]:
# dobr_grouped_uf.describe()

In [ ]:
# Salvar arquivo por UF para exportação
# dobr_grouped_uf.to_excel('exportar/dobr_grouped_uf_v2.xlsx', index=False)

In [ ]:
# heat_ds_treat2.columns

In [ ]:
# # Trazer dados por UF (setting e setting_average)

# # Realizar o merge entre heat_ e uf_sett
# heat_treat00 = pd.merge(heat_ds_treat2, dobr_grouped_uf, on=['UF',
#                                                                 'ANO_OBITO',
#                                                                 # 'SEXO',
#                                                                 # 'RACACOR'
#                                                                 ], how='left')

# # Exibir as primeiras linhas do dataframe resultante
# heat_treat00.head()

In [ ]:
# # Teste de aplicação de setting_average (médias por UF)
# filtered_data = heat_treat00[   (heat_treat00['ANO_OBITO'] == '_2022') &
#                                 (heat_treat00['UF'] == 'AC') 
#                                 # & (heat_treat00['SEXO'] == '_1') &
#                                 #(heat_treat00['RACACOR'] == '_4')
#                               ]
# filtered_data.head()

In [166]:
# heat_treat00.columns

Index(['CODMUNRES', 'ANO_OBITO', 'POP_MERGE', 'CONTADOR', 'pop_under74', 'UF',
       'mun_no', 'mort_ind_mun', 'obito_count_uf', 'pop_under74_uf',
       'mort_ind_uf'],
      dtype='object')

In [177]:
# heat_treat00.pop_under74_uf.sum()

74248156186.0

In [174]:
# obito_num = heat_treat00.CONTADOR.sum()
# pop_under74 = heat_treat00.pop_under74.sum()

# print(f"\033[33m ==> Número de mortes: {obito_num} \033[0m")
# print(f"\033[33m ==> Número de população abaixo de 75 anos é de {pop_under74} \033[0m")  


 ==> Número de mortes: 534729 
 ==> Número de população abaixo de 75 anos é de 194644474.0 


In [ ]:
# heat_template = row
# heat_template

##### Criar colunas do dataset

In [141]:
# Agrupar os dados por ano e calcular o número de óbitos e a população abaixo de 74 anos para cada ano
grouped_data = heatds_temp03.groupby('ANO_OBITO').agg({
    'CONTADOR': 'sum',
    'pop_under74': 'sum'
}).reset_index()

# Calcular set_avg_BR para cada ano
grouped_data['set_avg_BR'] = (grouped_data['CONTADOR'] / grouped_data['pop_under74']) * 100000

# Exibir os resultados
print(grouped_data[['ANO_OBITO', 'set_avg_BR']])

  ANO_OBITO  set_avg_BR
0     _2010  257.615194
1     _2020  269.469531


In [197]:
# Realizar o merge entre heat_ds_treat2 e grouped_data usando a coluna ANO_OBITO
heat_ds_treat2 = pd.merge(heat_ds_treat1, grouped_data[['ANO_OBITO', 'set_avg_BR']], on='ANO_OBITO', how='left')

# Exibir as primeiras linhas do dataframe resultante
heat_ds_treat2.head()

,CODMUNRES,ANO_OBITO,POP_MERGE,CONTADOR,pop_under74,UF,mun_no,mort_ind_mun,set_avg_BR
0,_110001,_2010,_2010_110001,58,23948.0,RO,Alta Floresta D'Oeste,242.191415,257.615194
1,_110001,_2019,_2022_110001,67,20742.0,RO,Alta Floresta D'Oeste,323.016103,258.919766
2,_110001,_2020,_2022_110001,68,20742.0,RO,Alta Floresta D'Oeste,327.837238,256.542023
3,_110001,_2021,_2022_110001,45,20742.0,RO,Alta Floresta D'Oeste,216.951114,266.310807
4,_110001,_2022,_2022_110001,64,20742.0,RO,Alta Floresta D'Oeste,308.552695,274.720874


In [198]:
# Gerar colunas sem informação ou com informação padronizada
# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heat_treat3 = heat_ds_treat2.copy() # caso seja agrupado por UF anteriormente

heat_treat3['setting'] = 'Brasil'
# heat_treat01['setting_average'] = set_avg_BR

heat_treat3['indicator_abbr'] = 'AVM'
heat_treat3['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heat_treat3['source'] = 'SIM/SUS & Censo/IBGE'
heat_treat3['favourable_indicator'] = 0
heat_treat3['indicator_scale'] = 100000
heat_treat3['ordered_dimension'] = 0
heat_treat3['subgroup_order'] = 0
heat_treat3['reference_subgroup'] = 0
heat_treat3['note'] = ''
heat_treat3['se'] = ''
heat_treat3['ci_lb'] = ''
heat_treat3['ci_ub'] = ''
heat_treat3['iso3'] = 'BRA'


heat_treat3.head()


,CODMUNRES,ANO_OBITO,POP_MERGE,CONTADOR,pop_under74,UF,mun_no,mort_ind_mun,set_avg_BR,setting,...,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3
0,_110001,_2010,_2010_110001,58,23948.0,RO,Alta Floresta D'Oeste,242.191415,257.615194,Brasil,...,0,100000,0,0,0,,,,,BRA
1,_110001,_2019,_2022_110001,67,20742.0,RO,Alta Floresta D'Oeste,323.016103,258.919766,Brasil,...,0,100000,0,0,0,,,,,BRA
2,_110001,_2020,_2022_110001,68,20742.0,RO,Alta Floresta D'Oeste,327.837238,256.542023,Brasil,...,0,100000,0,0,0,,,,,BRA
3,_110001,_2021,_2022_110001,45,20742.0,RO,Alta Floresta D'Oeste,216.951114,266.310807,Brasil,...,0,100000,0,0,0,,,,,BRA
4,_110001,_2022,_2022_110001,64,20742.0,RO,Alta Floresta D'Oeste,308.552695,274.720874,Brasil,...,0,100000,0,0,0,,,,,BRA


In [199]:
heat_treat3.columns

Index(['CODMUNRES', 'ANO_OBITO', 'POP_MERGE', 'CONTADOR', 'pop_under74', 'UF',
       'mun_no', 'mort_ind_mun', 'set_avg_BR', 'setting', 'indicator_abbr',
       'indicator_name', 'source', 'favourable_indicator', 'indicator_scale',
       'ordered_dimension', 'subgroup_order', 'reference_subgroup', 'note',
       'se', 'ci_lb', 'ci_ub', 'iso3'],
      dtype='object')

In [285]:
# # Trazer labels para sexo e raçacor
# heat_treat02 = heat_treat01.copy()

# heat_treat02['SEXO'] = heat_treat02['SEXO'].map({
#                                             '_1':'Homens',
#                                             '_2':'Mulheres'
#                                             })
# heat_treat02['RACACOR'] = heat_treat02['RACACOR'].map({
#                                             '_1':'Branca',
#                                             '_2':'Preta',
#                                             '_3':'Amarela',
#                                             '_4':'Parda',
#                                             '_5':'Indígena',
#                                             '_9':'Sem declaração'
#                                             })

# heat_treat02['subgroup'] = heat_treat02['SEXO'] + ' - raça/cor '  +  heat_treat02['RACACOR']


In [53]:
# heat_treat02['subgroup'].unique()

array(['Homens - raça/cor Parda', 'Mulheres - raça/cor Branca',
       'Mulheres - raça/cor Parda', 'Homens - raça/cor Branca',
       'Homens - raça/cor Indígena', 'Mulheres - raça/cor Indígena',
       'Homens - raça/cor Preta', 'Mulheres - raça/cor Preta',
       'Homens - raça/cor Amarela', 'Mulheres - raça/cor Amarela'],
      dtype=object)

In [286]:
# heat_treat02.columns

Index(['CODMUNRES', 'SEXO', 'RACACOR', 'ANO', 'POP_MERGE', 'obito_count',
       'pop_under74', 'UF', 'mun_nome', 'mort_ind_mun', 'obito_count_uf',
       'pop_under74_uf', 'mort_ind_uf', 'indicator_abbr', 'indicator_name',
       'source', 'favourable_indicator', 'indicator_scale',
       'ordered_dimension', 'subgroup_order', 'reference_subgroup', 'note',
       'se', 'ci_lb', 'ci_ub', 'iso3', 'subgroup'],
      dtype='object')

In [200]:
# Renomear colunas de heat_treat01
heat_treat4 = heat_treat3.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'UF':'dimension',
    'mun_no':'subgroup',
    'mort_ind_mun':'estimate',
    'set_avg_BR':'setting_average',
    # 'mort_ind_uf':'setting_average'
})

heat_treat4.head()

,CODMUNRES,date,POP_MERGE,CONTADOR,population,dimension,subgroup,estimate,setting_average,setting,...,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3
0,_110001,_2010,_2010_110001,58,23948.0,RO,Alta Floresta D'Oeste,242.191415,257.615194,Brasil,...,0,100000,0,0,0,,,,,BRA
1,_110001,_2019,_2022_110001,67,20742.0,RO,Alta Floresta D'Oeste,323.016103,258.919766,Brasil,...,0,100000,0,0,0,,,,,BRA
2,_110001,_2020,_2022_110001,68,20742.0,RO,Alta Floresta D'Oeste,327.837238,256.542023,Brasil,...,0,100000,0,0,0,,,,,BRA
3,_110001,_2021,_2022_110001,45,20742.0,RO,Alta Floresta D'Oeste,216.951114,266.310807,Brasil,...,0,100000,0,0,0,,,,,BRA
4,_110001,_2022,_2022_110001,64,20742.0,RO,Alta Floresta D'Oeste,308.552695,274.720874,Brasil,...,0,100000,0,0,0,,,,,BRA


In [201]:
heat_treat4.CONTADOR.sum()

2533528

In [202]:
heat_treat4.estimate.describe()

count    27817.000000
mean       268.207663
std         87.673481
min         12.640627
25%        211.782434
50%        263.208494
75%        317.028986
max        940.733772
Name: estimate, dtype: float64

In [203]:
heat_treat4['date'] = heat_treat4['date'].str.replace('_','').astype(int)
heat_treat4.head()

,CODMUNRES,date,POP_MERGE,CONTADOR,population,dimension,subgroup,estimate,setting_average,setting,...,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3
0,_110001,2010,_2010_110001,58,23948.0,RO,Alta Floresta D'Oeste,242.191415,257.615194,Brasil,...,0,100000,0,0,0,,,,,BRA
1,_110001,2019,_2022_110001,67,20742.0,RO,Alta Floresta D'Oeste,323.016103,258.919766,Brasil,...,0,100000,0,0,0,,,,,BRA
2,_110001,2020,_2022_110001,68,20742.0,RO,Alta Floresta D'Oeste,327.837238,256.542023,Brasil,...,0,100000,0,0,0,,,,,BRA
3,_110001,2021,_2022_110001,45,20742.0,RO,Alta Floresta D'Oeste,216.951114,266.310807,Brasil,...,0,100000,0,0,0,,,,,BRA
4,_110001,2022,_2022_110001,64,20742.0,RO,Alta Floresta D'Oeste,308.552695,274.720874,Brasil,...,0,100000,0,0,0,,,,,BRA


In [204]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  

# Reorder the columns in the DataFrame
heat_treat5 = heat_treat4[column_order]

# Transformar estimate e setting_average em int
heat_treat5['estimate'] = heat_treat5['estimate'].astype(int)
heat_treat5['setting_average'] = heat_treat5['setting_average'].astype(int)


heat_treat5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27817 entries, 0 to 27816
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   setting               27817 non-null  object 
 1   date                  27817 non-null  int64  
 2   source                27817 non-null  object 
 3   indicator_abbr        27817 non-null  object 
 4   indicator_name        27817 non-null  object 
 5   dimension             27817 non-null  object 
 6   subgroup              27817 non-null  object 
 7   estimate              27817 non-null  int64  
 8   se                    27817 non-null  object 
 9   ci_lb                 27817 non-null  object 
 10  ci_ub                 27817 non-null  object 
 11  population            27817 non-null  float64
 12  note                  27817 non-null  object 
 13  setting_average       27817 non-null  int64  
 14  iso3                  27817 non-null  object 
 15  favourable_indicato

/tmp/ipykernel_33777/2301066262.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_treat5['estimate'] = heat_treat5['estimate'].astype(int)
/tmp/ipykernel_33777/2301066262.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_treat5['setting_average'] = heat_treat5['setting_average'].astype(int)


In [205]:
### Arquivo pronto para exportação
heat_treat5.to_excel('exportar/ds_4val_v2.xlsx', index=False)

In [206]:
heat_treat5.shape[0]

27817

In [181]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('downloads_outros/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('exportar/ds_4val.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('exportar/heat_plus_template_ds_4val.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados